In [5]:
import os
import pandas as pd

In [6]:
# pip install 'transformers[torch]'
# pip install 'transformers[tf-cpu]'
# pip install tensorflow

In [7]:
#pip install tf-keras

In [8]:
all_labels = ['in_silico','organs','other','human','in_vivo','invertebrate','primary_cells','immortal_cell_line']

#######################################
### --------- Import text --------- ###
def read_text(pmid,docs_dir):
	txt_file = os.path.join(docs_dir,pmid+".txt")
	with open(txt_file, "r") as text_file:
		text = text_file.read()
		text = text.replace("\n"," ")
		text = text.replace("\t"," ")
	return text

#######################################
### -------- Import Splits -------- ###
def import_splits(docs_dir,train_dev_test_dir,filename):
	tsv_file = filename[0:-5]+".tsv"
	with open(os.path.join(tsv_file), "w") as writer:
		line = "PMID"
		for label in all_labels:
			line += "\t"+label
		line += "\tTEXT\n"
		writer.write(line)
		skipped = []
		doc_index = 0
		with open(os.path.join(train_dev_test_dir,filename), "r") as reader:
			lines = reader.readlines()
			for line in lines:
				pmid, str_labels = line.strip().split("\t")
				labels = str_labels.split(",")
				text = read_text(pmid,docs_dir)
				# exclude documents w/o text
				if len(text)==0:
					skipped.append(doc_index)
					continue
				#print(pmid,labels,text)
				line = pmid
				for label in all_labels:
					if label in labels:
						line += "\t1"
					else:
						line += "\t0"
				line += "\t"+text+"\n"
				writer.write(line)
				doc_index += 1
	writer.close()
	return tsv_file, skipped

############################################
### --------- Pre-process data --------- ###
def pre_process_data(docs_dir,train_dev_test_dir,filename):
	# Import splits
	tsv_file, skipped = import_splits(docs_dir,train_dev_test_dir,filename)
	# Import data from tsv
	data = pd.read_csv(tsv_file,sep='\t')
	# Select required columns
	filters = []
	for label in all_labels:
		filters.append(label)
	filters.append('TEXT')
	data = data[filters]
	print(data)
	# Set your model output as categorical and save in new label col
	for label in all_labels:
		if label in data:
			data[label+'_label'] = pd.Categorical(data[label])
	# Transform your output to numeric
	for label in all_labels:
		if label in data:
			data[label] = data[label+'_label'].cat.codes
	return data, tsv_file, skipped

In [9]:
DOCS_DIR = "pubmed_docs"
TRAIN_DEV_TEST_DIR = "corpus_annotations"

In [10]:
range_values = range(0,10)
for split in range_values:
    print("*** ",split," ***")
    data_train, tsv_file_train, skipped_train = pre_process_data(DOCS_DIR,TRAIN_DEV_TEST_DIR,'train'+str(split)+'.txt')
    data_dev, tsv_file_dev, skipped_dev = pre_process_data(DOCS_DIR,TRAIN_DEV_TEST_DIR,'dev'+str(split)+'.txt')

***  0  ***
      in_silico  organs  other  human  in_vivo  invertebrate  primary_cells  \
0             0       0      0      0        0             0              0   
1             0       0      0      0        0             0              0   
2             0       0      0      0        0             0              0   
3             0       0      0      0        0             0              0   
4             0       0      0      0        0             0              0   
...         ...     ...    ...    ...      ...           ...            ...   
1287          0       0      0      0        1             0              0   
1288          0       0      0      0        1             0              0   
1289          0       0      0      0        1             0              0   
1290          0       0      0      0        1             0              0   
1291          0       0      0      0        1             0              0   

      immortal_cell_line               

In [17]:
!python --version

Python 3.11.7


In [19]:
from transformers import TFBertModel, BertTokenizerFast, BertModel
from tensorflow.keras.layers import Input, Dropout, Dense
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.utils import to_categorical

/opt/anaconda3/envs/transformers_tf_env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [25]:
import transformers
import tensorflow
print("Transformers version:", transformers.__version__)
print("TF version:", tensorflow.__version__)


Transformers version: 4.33.3
TF version: 2.13.0


In [87]:
from transformers import TFBertModel, BertTokenizerFast, BertModel
from tensorflow.keras.layers import Input, Dropout, Dense
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.optimizers import legacy as legacy_optimizers

#######################################
### -------- Setup BioBERT -------- ###
def setup_bert(model_name='dmis-lab/biobert-v1.1'):
    # Max length of tokens
    #	max_length = 128
    max_length = 256
    # Load BioBERT tokenizer
    tokenizer = BertTokenizerFast.from_pretrained(model_name)
    # Load the Transformers BERT model
    transformer_model = TFBertModel.from_pretrained(model_name, from_pt=True)
    
    config = transformer_model.config
    
    return transformer_model, config, max_length, tokenizer

#######################################
### ------- Build the model ------- ###

def build_model(transformer_model,config,max_length,data):
	# TF Keras documentation: https://www.tensorflow.org/api_docs/python/tf/keras/Model# Load the MainLayer
	bert = transformer_model.layers[0]
	# Build your model input
	input_ids = Input(shape=(max_length,), name='input_ids', dtype='int32')
	inputs = {'input_ids': input_ids}
	# Load the Transformers BERT model as a layer in a Keras model
	bert_model = bert(inputs)[1]
	dropout = Dropout(config.hidden_dropout_prob, name='pooled_output')
	pooled_output = dropout(bert_model, training=False)
	# Then build your model output
	# one per output
	outputs = {}
	for label in all_labels:
		label_output = Dense(units=len(data[label+'_label'].value_counts()), kernel_initializer=TruncatedNormal(stddev=config.initializer_range), name=label)(pooled_output)
		outputs[label] = label_output
	# And combine it all in a model object
	model = Model(inputs=inputs, outputs=outputs, name='BERT_MultiLabel_MultiClass')
	# Take a look at the model
	model.summary()
	return model

#######################################
### ------- Train the model ------- ###
def train_model(model_name, model,data_train,data_dev,max_length,tokenizer,learning_rate,batch_size,epochs):
    # Set an optimizer
    optimizer = legacy_optimizers.Adam(
        learning_rate,
        epsilon=1e-08,
        decay=0.01,
        clipnorm=1.0)
    # Set loss and metrics
    loss = {}
    for label in all_labels:
        loss[label] = CategoricalCrossentropy(from_logits = True)
    metric = {}
    for label in all_labels:
        metric[label] = CategoricalAccuracy('accuracy')
    # Compile the model
    model.compile(
        optimizer = optimizer,
        loss = loss, 
        metrics = metric)
    # trainig data
    x_train, y_train = prepare_x_y_dev_test(data_train,max_length,tokenizer)
    # validation data
    x_val, y_val = prepare_x_y_dev_test(data_dev,max_length,tokenizer)
    # Fit the model
    history = model.fit(
        x_train, y_train,
        validation_data=(x_val,y_val),
        batch_size=batch_size,
        epochs=epochs
    )
    model_name_clean = model_name.replace("/","_")
    model.save(model_name_clean + "_model.h5")

def prepare_x_y_dev_test(data,max_length,tokenizer):
	test_ys = {}
	for label in all_labels:
		test_y = to_categorical(data[label])	
		test_ys[label] = test_y
	test_x = tokenizer(
	    text=data['TEXT'].to_list(),
    	add_special_tokens=True,
	    max_length=max_length,
	    truncation=True,
	    padding=True, 
	    return_tensors='tf',
	    return_token_type_ids = False,
	    return_attention_mask = False,
	    verbose = True)
	x={'input_ids': test_x['input_ids']}
	y = {}
	for label in all_labels:
		y[label] = test_ys[label]
	return x, y

####################################
### ----- Train the model ------ ###
def train_bert_goldhamster2(model_name, docs_dir,train_dev_test_dir,train_file,dev_file):
	# import data
	data_train, tsv_file_train, skipped_train = pre_process_data(docs_dir,train_dev_test_dir,train_file)
	data_dev, tsv_file_dev, skipped_dev = pre_process_data(docs_dir,train_dev_test_dir,dev_file)
	# set up model
	transformer_model, config, max_length, tokenizer = setup_bert(model_name) 
	# build model
	model = build_model(transformer_model,config,max_length,data_train)
	# train model
	train_model(model_name, model,data_train,data_dev,max_length,tokenizer,learning_rate,batch_size,epochs)

###########################################
### ----- Predict with the model ------ ###
# Ready test data
def predict_with_model(model_name,docs_dir,train_dev_test_dir,test_file,out_file):
    # import data
    data_test, tsv_file_test, skipped_test = pre_process_data(docs_dir,train_dev_test_dir,test_file)
    # set up model
    transformer_model, config, max_length, tokenizer = setup_biobert()
    # prepare test data
    x, y = prepare_x_y_dev_test(data_test,max_length,tokenizer)
    # Load model
    model_name_clean = model_name.replace("/","_")
    model = load_model(model_name_clean + '_model.h5')
    # Run predictions
    predictions = model.predict(x)
    print(model.summary())
    print_predictions(predictions,train_dev_test_dir,test_file,out_file,skipped_test)

def print_predictions(predictions,train_dev_test_dir,test_file,out_file,skipped_test):
	#print(len(predictions[label]))	
	with open(os.path.join(train_dev_test_dir,out_file), "w") as writer:
		with open(os.path.join(train_dev_test_dir,test_file), "r") as reader:
			lines = reader.readlines()
			doc_index = 0
			for line in lines:
				pmid, str_labels = line.strip().split("\t")
				list_labels = []
				if doc_index not in skipped_test:
					for label in predictions:
						arr_pred = predictions[label][doc_index]
						#print(arr_pred)
						if arr_pred[1]>arr_pred[0]:
							list_labels.append(label)
					doc_index += 1
				#print(pmid,list_labels)
				writer.write(pmid+"\t"+','.join(list_labels)+"\n")
		writer.close()

def train_cross_validation(model_name, docs_dir,train_dev_test_dir,name):
    print(train_dev_test_dir)
    range_values = range(0,10)
    model_name_clean = model_name.replace("/", "_")
    for split in range_values:
        print("*** ",split," ***")
        train_bert_goldhamster2(model_name, docs_dir,train_dev_test_dir,'train'+str(split)+'.txt','dev'+str(split)+'.txt')
        predict_with_model(model_name,docs_dir,train_dev_test_dir,'test'+str(split)+'.txt', model_name_clean + '_preds_' +str(split)+'_'+name+'.txt')

def train_one_experiment(model_name, docs_dir,train_dev_test_dir,split,name):
    model_name_clean = model_name.replace("/", "_")
    train_bert_goldhamster2(model_name, docs_dir,train_dev_test_dir,'train'+str(split)+'.txt','dev'+str(split)+'.txt')
    predict_with_model(model_name,docs_dir,train_dev_test_dir,'test'+str(split)+'.txt', model_name_clean + '_preds_'+str(split)+'_'+name+'.txt')

In [89]:
model_name = 'dmis-lab/biobert-v1.1'
learning_rate = 1e-04
batch_size = 32
epochs = 1
best_split = 1

# folders
train_dev_test_dir = TRAIN_DEV_TEST_DIR
docs_dir = DOCS_DIR
train_one_experiment(model_name,docs_dir,train_dev_test_dir,best_split,"goldhamster")
#
#train_cross_validation(model_name,docs_dir,train_dev_test_dir,"goldhamster")


      in_silico  organs  other  human  in_vivo  invertebrate  primary_cells  \
0             0       0      0      0        0             0              0   
1             0       0      0      0        1             0              0   
2             0       0      0      0        0             0              0   
3             0       0      0      0        1             0              0   
4             0       0      0      0        0             0              0   
...         ...     ...    ...    ...      ...           ...            ...   
1293          0       0      0      0        1             0              0   
1294          0       0      0      0        1             0              0   
1295          0       0      0      0        1             0              0   
1296          0       0      0      0        1             0              0   
1297          0       0      0      0        1             0              0   

      immortal_cell_line                           

/opt/anaconda3/envs/transformers_tf_env/lib/python3.8/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
All PyTorch model weights were used when initializing TFBertModel.

All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "BERT_MultiLabel_MultiClass"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_ids (InputLayer)      [(None, 256)]                0         []                            
                                                                                                  
 bert (TFBertMainLayer)      TFBaseModelOutputWithPooli   1083102   ['input_ids[0][0]']           
                             ngAndCrossAttentions(last_   72                                      
                             hidden_state=(None, 256, 7                                           
                             68),                                                                 
                              pooler_output=(None, 768)                                           
                             , past_key_values=None, hi                  

/opt/anaconda3/envs/transformers_tf_env/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


    in_silico  organs  other  human  in_vivo  invertebrate  primary_cells  \
0           0       0      0      0        0             0              0   
1           0       0      0      0        0             0              0   
2           0       0      0      0        1             0              0   
3           0       0      0      0        0             0              0   
4           0       0      0      0        0             0              0   
..        ...     ...    ...    ...      ...           ...            ...   
65          0       0      0      0        1             0              0   
66          0       0      0      0        1             0              0   
67          0       0      0      0        1             0              0   
68          0       0      0      0        1             0              0   
69          0       0      0      0        1             0              0   

    immortal_cell_line                                               TEXT  

/opt/anaconda3/envs/transformers_tf_env/lib/python3.8/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
All PyTorch model weights were used when initializing TFBertModel.

All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


3/3 [==============================] - 7s 2s/step
Model: "BERT_MultiLabel_MultiClass"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_ids (InputLayer)      [(None, 256)]                0         []                            
                                                                                                  
 bert (TFBertMainLayer)      TFBaseModelOutputWithPooli   1083102   ['input_ids[0][0]']           
                             ngAndCrossAttentions(last_   72                                      
                             hidden_state=(None, 256, 7                                           
                             68),                                                                 
                              pooler_output=(None, 768)                                           
                       